In [ ]:
!pip install timm

In [ ]:
import torch
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2
import sklearn
from torch.utils.data import DataLoader, random_split, ConcatDataset, Subset, Dataset
from torchvision.datasets.folder import default_loader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import os
from google.colab import drive
import concurrent.futures
import time
import random
import csv
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import AdamW
import timm

In [ ]:
# Mount Google Drive for persistent storage
drive.mount('/content/drive')

In [ ]:
NUM_NODES = 1
NUM_GLOBAL_EPOCHS = 5
NUM_LOCAL_EPOCHS = 5
BATCH_SIZE = 64

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dirs, class_labels, transform=None):
        self.transform = transform
        self.samples = []
        self.labels = set()  # Used to store unique tag sets

        # Loop through each category's directory and read the files
        for i, dirs in enumerate(root_dirs):
            for dir_path in dirs:
                for img_name in os.listdir(dir_path):
                    img_path = os.path.join(dir_path, img_name)
                    self.samples.append((img_path, i))  # Use index as label
                    self.labels.add(class_labels[i])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path, label = self.samples[index]
        sample = default_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, label

In [ ]:
def load_data():
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    # Use the CustomDataset
    root_dirs = [
    ['/content/drive/My Drive/Swarm_Learning/Data/Blood cell Cancer [ALL]/Benign'],
    ['/content/drive/My Drive/Swarm_Learning/Data/Blood cell Cancer [ALL]/[Malignant] Pre-B'],
    ['/content/drive/My Drive/Swarm_Learning/Data/Blood cell Cancer [ALL]/[Malignant] Pro-B'],
    ['/content/drive/My Drive/Swarm_Learning/Data/Blood cell Cancer [ALL]/[Malignant] early Pre-B']
    ]
    class_labels = ['Benign', 'Malignant_Pre-B', 'Malignant_Pro-B', 'Malignant_early Pre-B']
    dataset = CustomDataset(root_dirs, class_labels, transform=transform)
    num_classes = len(dataset.labels)
    print("Number of samples in the dataset:", len(dataset))
    print("Detected number of classes:", num_classes)

    # Split the dataset into training and testing sets (80% train, 20% test)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    return train_dataset, test_dataset, num_classes


In [ ]:
def split_data_nodes(dataset, num_nodes):
  # Determine the sizes of each split
  dataset_size = len(dataset)
  indices = list(range(dataset_size))
  np.random.shuffle(indices)

  split_sizes = np.random.randint(1, dataset_size // num_nodes + 1, size=num_nodes - 1)
  split_sizes = np.append(split_sizes, dataset_size - split_sizes.sum())
  np.random.shuffle(split_sizes)

  # Create random splits
  subsets = []
  start = 0
  for size in split_sizes:
      subset_indices = indices[start:start + size]
      subsets.append(subset_indices)
      start += size


  # Apply the splits to the dataset
  dataset_splits = [torch.utils.data.Subset(dataset, subset) for subset in subsets]

  # Calculate the size weightings for each subset
  weightings = [len(subset) / dataset_size for subset in dataset_splits]



  print("Data has been split into " + str(num_nodes) + " nodes")
  # Print the size of each data subset
  for i, subset in enumerate(dataset_splits):
      print(f'Size of subset {i+1}: {len(subset)}')


  return dataset_splits, dataset, weightings

In [ ]:
def create_data_loaders(subset):
    # Split into train and validation (adjust validation split as needed)
    val_size = int(0.2 * len(subset))
    train_size = len(subset) - val_size
    train_subset, val_subset = torch.utils.data.random_split(subset, [train_size, val_size])

    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    print("Data Loader Created")
    return train_loader, val_loader

In [ ]:
class SwinTransformerModel(nn.Module):
    def __init__(self, num_classes=4):
        super(SwinTransformerModel, self).__init__()
        self.swin_transformer = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)

        # Freeze all parameters of the pre-trained model
        for param in self.swin_transformer.parameters():
            param.requires_grad = False

        # Get the number of input features for the last layer
        num_features = self.swin_transformer.head.in_features
        self.swin_transformer.head = nn.Sequential(
            nn.Dropout(0.5),  # Adding Dropout Layers to Reduce Overfitting
            nn.Linear(num_features, 512),  # Top level fully connected layer
            nn.ReLU(),  # Activation function
            nn.Linear(512, num_classes)  # Output layer
        )

        # Ensure that only the parameters of the newly added fully connected layer are updated
        for param in self.swin_transformer.head.parameters():
            param.requires_grad = True

        for name, param in self.swin_transformer.named_parameters():
            if name in ['layer4.2.conv3.weight', 'layer4.2.bn3.weight', 'layer4.2.bn3.bias']:
                param.requires_grad = True

        # Add a global average pooling layer to handle the spatial dimensions
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.swin_transformer.forward_features(x)  # Extract features

        # Adjust the dimension order
        x = x.permute(0, 3, 1, 2)  # From [32, 7, 7, 768] to [32, 768, 7, 7]

        # Apply global average pooling
        x = self.global_avg_pool(x)  # From [32, 768, 7, 7] to [32, 768, 1, 1]

        x = torch.flatten(x, 1)  # Flatten from [32, 768, 1, 1] to [32, 768]
        x = self.swin_transformer.head(x)  # Apply fully connected layer

        return x


In [ ]:
def create_final_data_test_loader(final_testing):
  final_model_test_loader = torch.utils.data.DataLoader(final_testing, batch_size=BATCH_SIZE, shuffle=False)
  return final_model_test_loader

In [ ]:
def model_parameters_to_string(model):
    params_str = "Model Parameters:\n"
    for name, param in model.named_parameters():
        params_str += f"Name: {name}, Shape: {param.shape}\n"
        params_str += f"{param.data}\n"
        params_str += "-" * 50 + "\n"
    return params_str

In [ ]:

class Node:
    def __init__(self, node_id, train_loader: DataLoader, test_loader: DataLoader, weighting, aggregation_methodology):
        """
        Initialize the Node with training and testing DataLoaders.

        Parameters:
        train_loader (DataLoader): DataLoader for training data.
        test_loader (DataLoader): DataLoader for testing data.
        """
        self.__train_loader = train_loader
        self.test_loader = test_loader
        self.node_id = node_id
        self.global_parameters = []
        self.accuracy = None
        self.model = SwinTransformerModel() ##num_classes = 4
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.AdamW(self.model.parameters(), lr=0.001, weight_decay=0.01)
        self.accuracy_scores = []
        self.current_accuracy = 0
        self.accuracy_scores = []
        self.f1_scores = []
        self.precision_scores = []
        self.recall_scores = []
        self.weighting = weighting
        self.aggregation_methodology = aggregation_methodology
        self.balance_score = self.get_balance_score()

    def get_model(self):
      return self.model

    def get_balance_score(self):
        class_counts = np.zeros(4, dtype=int)  # 4 classes
        for _, label in self.__train_loader:
            class_counts[label] += 1
        total_samples = sum(class_counts)
        if total_samples == 0:
            return 0
        class_proportions = np.array(class_counts) / total_samples
        score = 1 - np.std(class_proportions)
        return score

    def get_final_scores(self):

      """
      Convert performance metrics lists into a pandas DataFrame.

      Parameters:
      accuracy_scores (list): List of accuracy scores.
      precision_scores (list): List of precision scores.
      recall_scores (list): List of recall scores.
      f1_scores (list): List of F1 scores.

      Returns:
      pd.DataFrame: DataFrame containing the metrics.
      """
      # Create a dictionary from the lists
      metrics_dict = {
          'Accuracy': self.accuracy_scores,
          'Precision': self.precision_scores,
          'Recall': self.recall_scores,
          'F1 Score': self.f1_scores
      }

      # Convert the dictionary into a DataFrame
      df = pd.DataFrame(metrics_dict)

      return df

    def get_accuracy_score(self):
      return self.current_accuracy

    def set_accuracy_score(self, accuracy):
      self.current_accuracy = accuracy

    def get_node_id(self):
      return self.node_id

    def get_test_loader(self) -> DataLoader:
        """
        Return the testing DataLoader.

        Returns:
        DataLoader: The testing DataLoader.
        """
        return self.test_loader

    def broadcast_parameters(self):
        """
        Return the model parameters.

        Returns:
        dict: Dictionary of model parameters.
        """
        return_list = []
        for param in self.model.parameters():
          return_list.append(param.clone())

        return_list.insert(0, self.current_accuracy)
        return_list.insert(0, self.weighting)
        return_list.insert(0, self.balance_score)

        return return_list


    def gather_parameters(self, params):
        print("Gathered Parameters for Node " + str(self.node_id))
        self.global_parameters.append(params)
        print("Length of Gathered Parameters for Node " + str(self.node_id) + " : " + str(len(self.global_parameters)))

    def average_parameters(self):
      # Initialize a list to store the summed parameters
      averaged_params = []

      params = self.global_parameters
      for inner_list in params:
        inner_list.pop(0)
        inner_list.pop(0)
        inner_list.pop(0)
      # Iterate over the parameters of the first model to initialize the structure of the average
      for param in params[0]:
          # Start with a tensor of zeros of the same shape as the parameter
          averaged_params.append(torch.zeros_like(param))

      # Sum the parameters from each model
      for model_params in params:
          for i, param in enumerate(model_params):
              averaged_params[i] += param

      # Divide by the number of models to get the average
      num_models = len(params)
      for i in range(len(averaged_params)):
          averaged_params[i] /= num_models

      return averaged_params


    def size_average_parameters(self):
      averaged_params = []
      print("Size of global params: " + str(len(self.global_parameters)))
      params = self.global_parameters
      weightings = []

      for param in params:
        param.pop(0) ##Remove balance Score
        weighting = param.pop(0) ##Get weighting
        param.pop(0) ## Remove accuracy score
        weightings.append(weighting)

      for param in params[0]:
          # Start with a tensor of zeros of the same shape as the parameter
          averaged_params.append(torch.zeros_like(param))

      # Sum the parameters with their corresponding weights
      total_weight = 0
      for model_params, weight in zip(params, weightings):
          if len(model_params) != len(averaged_params):
              raise ValueError("Mismatch in the number of parameters between models.")
          for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
              if param.shape != avg_param.shape:
                  raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
              averaged_params[i] += param * weight
          total_weight += weight

      # Normalize by the total weight
      if total_weight > 0:
          for i in range(len(averaged_params)):
              averaged_params[i] /= total_weight

      return averaged_params



    def accuracy_average_parameters(self):
      averaged_params = []
      print("Size of global params: " + str(len(self.global_parameters)))
      params = self.global_parameters
      accuracies = []

      for param in params:
        param.pop(0) ##Get rid of balance score
        param.pop(0) ##Get rid of Weighting
        accuracy = param.pop(0)
        print("Accuracy: " + str(accuracy))
        accuracies.append(accuracy)

      for param in params[0]:
          # Start with a tensor of zeros of the same shape as the parameter
          averaged_params.append(torch.zeros_like(param))

          # Sum the parameters with their corresponding weights
      total_accuracies = 0
      for model_params, accuracy in zip(params, accuracies):
          if len(model_params) != len(averaged_params):
              raise ValueError("Mismatch in the number of parameters between models.")
          for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
              if param.shape != avg_param.shape:
                  raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
              averaged_params[i] += param * accuracy
          total_accuracies += accuracy

      # Normalize by the total weight
      if total_accuracies > 0:
          print("Total Accuracy: " + str(total_accuracies))
          for i in range(len(averaged_params)):
              averaged_params[i] /= total_accuracies

      return averaged_params

    def selective_accuracy_average_parameters(self):
      averaged_params = []
      print("Size of global params: " + str(len(self.global_parameters)))
      params = self.global_parameters
      accuracies = []

      for param in params:
        param.pop(0) ##Get rid of balance score
        param.pop(0) ##Get rid of Weighting
        accuracy = param.pop(0)
        print("Accuracy: " + str(accuracy))
        accuracies.append(accuracy)

      # Identify the index of the node with the lowest accuracy
      min_accuracy_index = accuracies.index(min(accuracies))
          # Exclude the parameters of the node with the lowest accuracy
      filtered_params = [p for i, p in enumerate(params) if i != min_accuracy_index]
      filtered_accuracies = [a for i, a in enumerate(accuracies) if i != min_accuracy_index]

      # Check if we have remaining nodes after exclusion
      if not filtered_params:
          raise ValueError("No parameters left to average after excluding the lowest accuracy node.")

      # Initialize averaged_params with zeros
      for param in filtered_params[0]:
          averaged_params.append(torch.zeros_like(param))

      # Sum the parameters with their corresponding weights
      total_accuracies = 0
      for model_params, accuracy in zip(filtered_params, filtered_accuracies):
          if len(model_params) != len(averaged_params):
              raise ValueError("Mismatch in the number of parameters between models.")
          for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
              if param.shape != avg_param.shape:
                  raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
              averaged_params[i] += param * accuracy
          total_accuracies += accuracy

      # Normalize by the total weight
      if total_accuracies > 0:
          print("Total Accuracy: " + str(total_accuracies))
          for i in range(len(averaged_params)):
              averaged_params[i] /= total_accuracies

      return averaged_params


    def balance_average_parameters(self):
      averaged_params = []
      print("Size of global params: " + str(len(self.global_parameters)))
      params = self.global_parameters
      balance_scores = []

      for param in params:
        balance_score = param.pop(0) ##Store balance score
        param.pop(0) ##Get rid of Weighting
        param.pop(0) ##Get rid of Accuracy
        print("Balance Score: " + str(balance_score))
        balance_scores.append(balance_score)

      for param in params[0]:
          # Start with a tensor of zeros of the same shape as the parameter
          averaged_params.append(torch.zeros_like(param))

      # Sum the parameters with their corresponding weights
      total_balance_scores = 0
      for model_params, balance_score in zip(params, balance_scores):
          if len(model_params) != len(averaged_params):
              raise ValueError("Mismatch in the number of parameters between models.")
          for i, (param, avg_param) in enumerate(zip(model_params, averaged_params)):
              if param.shape != avg_param.shape:
                  raise RuntimeError(f"Shape mismatch: {param.shape} != {avg_param.shape}")
              averaged_params[i] += param * balance_score
          total_balance_scores += balance_score

      # Normalize by the total weight
      if total_balance_scores > 0:
          print("Total Balance Scores: " + str(total_balance_scores))
          for i in range(len(averaged_params)):
              averaged_params[i] /= total_balance_scores

      return averaged_params




    def aggregate_parameters(self):
      if self.aggregation_methodology == "average":
        averaged_params = self.average_parameters()
      elif self.aggregation_methodology == "size_average":
        averaged_params = self.size_average_parameters()
      elif self.aggregation_methodology == "accuracy_average":
        averaged_params = self.accuracy_average_parameters()
      elif self.aggregation_methodology == "balance_average":
        averaged_params = self.balance_average_parameters()
      elif self.aggregation_methodology == "selective_accuracy_average":
        averaged_params = self.selective_accuracy_average_parameters()
      else:
        raise ValueError("Invalid aggregation methodology. Use 'average' or 'size_average' or 'accuracy_average'.")
      print("Length of Averaged Params for node " + str(self.node_id) + " : " + str(len(averaged_params)))
      self.set_parameters(averaged_params)

      self.global_parameters = []


    def set_parameters(self, parameters):

        """
        Set the model parameters.

        Parameters:
        parameters (dict): Dictionary of model parameters.
        """
        # Convert the list of averaged parameters back into the correct format for the model
        print("Setting Parameters for node: " + str(self.node_id))
        new_state_dict = dict(zip(self.model.state_dict().keys(), parameters))
        self.model.load_state_dict(new_state_dict)
        print("Set Parameters for node " + str(self.node_id))



    def train(self):
      # Move model to the specified device
      self.model.to(device)

      for epoch in range(NUM_LOCAL_EPOCHS):

        self.model.train()
        running_loss = 0.0

        for images, labels in self.__train_loader:
            # Move images and labels to the specified device
            images, labels = images.to(device), labels.to(device)

            # Flatten the labels tensor to be 1D
            if len(labels.shape) == 2:
                labels = labels.squeeze(1)  # Remove the extra dimension

            # Check for label values out of bounds
            if labels.max() >= 4:
                raise ValueError(f"Label value {labels.max()} is out of bounds for the number of classes {4}.")

            # Ensure labels are 1D
            if len(labels.shape) != 1:
                raise ValueError("Labels tensor is not 1D. It should be of shape [batch_size].")

            self.optimizer.zero_grad()
            outputs = self.model(images)

            # Ensure labels are of type long
            labels = labels.long()

            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
      # Print the average loss for this epoch
        print(f"Epoch [{epoch+1}/{NUM_LOCAL_EPOCHS}], Training Loss: {running_loss / len(self.__train_loader)}")




    def evaluate(self):
        self.model.eval()  # Set the model to evaluation mode

        all_labels = []
        all_predictions = []

        with torch.no_grad():  # No need to calculate gradients during evaluation
            for images, labels in self.test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        # Convert lists to numpy arrays for sklearn metrics
        all_labels = np.array(all_labels).flatten()
        all_predictions = np.array(all_predictions)

        # Calculate metrics
        accuracy = np.sum(all_predictions == all_labels) / len(all_labels)
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')
        f1 = f1_score(all_labels, all_predictions, average='weighted')

        self.accuracy_scores.append(accuracy)
        self.precision_scores.append(precision)
        self.recall_scores.append(recall)
        self.f1_scores.append(f1)
        self.set_accuracy_score(accuracy)
        return accuracy, precision, recall, f1




In [ ]:
def train_and_communicate(node, nodes):
    # Train the node
    node.train()
    print("HELLO")
    metrics = node.evaluate()
    print("Metrics for Node " + str(node.get_node_id()) + " : " + str(metrics))
    # Send parameters to all other nodes
    for other_node in nodes:
      other_node.gather_parameters(node.broadcast_parameters())
      print("NODE " + str(node.get_node_id()) + "  HAS BROADCASTED ")

    return

In [ ]:
def evaluate(model, test_loader):
  model.eval()  # Set the model to evaluation mode

  all_labels = []
  all_predictions = []

  with torch.no_grad():  # No need to calculate gradients during evaluation
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

  # Convert lists to numpy arrays for sklearn metrics
  all_labels = np.array(all_labels).flatten()
  all_predictions = np.array(all_predictions)

  # Print the shapes of the arrays
  print(f"Shape of all_labels: {all_labels.shape}")
  print(f"Shape of all_predictions: {all_predictions.shape}")


  print(f"Unique labels: {np.unique(all_labels)}")
  print(f"Unique predictions: {np.unique(all_predictions)}")


  print(f"Sample Labels: {all_labels[:10]}")
  print(f"Sample Predictions: {all_predictions[:10]}")

  print("Num Correct: " + str(np.sum(all_predictions == all_labels)))

  # Calculate metrics
  accuracy = np.sum(all_predictions == all_labels) / len(all_labels)
  precision = precision_score(all_labels, all_predictions, average='weighted')
  recall = recall_score(all_labels, all_predictions, average='weighted')
  f1 = f1_score(all_labels, all_predictions, average='weighted')

  return accuracy, precision, recall, f1


In [ ]:
# Function to write the metrics to a CSV file
def write_metrics_to_csv(filename, metrics_list):
    file_exists = os.path.isfile(filename)

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        # Write the header only if the file is being created
        if not file_exists:
            writer.writerow(['Aggregation Method', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

        # Write the data
        writer.writerow(metrics_list)

    print(f'Metrics written to {filename}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
aggregation_methodologies = ['average', 'size_average', 'balance_average', 'accuracy_average', 'selective_accuracy_average']
# aggregation_methodology = 'balance_average'
for aggregation_methodology in aggregation_methodologies:
  print("Aggregation Methodology: " + aggregation_methodology)
  for i in range(30):

      dataset, testing_set, num_classes = load_data()

      #split data into final testing and data for nodes
      subsets, dataset, weightings = split_data_nodes(dataset, 4)

      #create the final test loader and set it
      final_test_loader = create_final_data_test_loader(testing_set)

      nodes = []
      for i in range(0,NUM_NODES):
        train_loader, val_loader =  create_data_loaders(subsets[i])
        node = Node(i, train_loader, val_loader, weightings[i], aggregation_methodology)
        nodes.append(node)


      # Training and aggregation process
      for _ in range(NUM_GLOBAL_EPOCHS):  # Number of training rounds
          print("Round " + str(_))
          with concurrent.futures.ThreadPoolExecutor() as executor:
              executor.map(train_and_communicate, nodes, [nodes]*len(nodes))

          # Aggregation phase
          for node in nodes:
            updated_params = node.aggregate_parameters()

          time.sleep(1)  # Simulate time between rounds


      node = nodes[0]
      accuracy, precision, recall, f1 = evaluate(node.get_model(), final_test_loader)
      filename = '/content/drive/My Drive/Swarm_Learning/bloodcancer.csv'
      metrics_list = [aggregation_methodology, accuracy, precision, recall, f1]
      write_metrics_to_csv(filename, metrics_list)



